# Closed-Loop Evaluation
In this notebook you are going to evaluate a CNN-based policy to control the SDV with a protocol named *closed-loop* evaluation.

**Note: this notebook assumes you've already run the [training notebook](./train.ipynb) and stored your model successfully.**

## What is closed-loop evaluation?
In closed-loop evaluation the model is in **full control of the SDV**. At each time step, we predict the future trajectory and then move the AV to the first of the model's predictions. 

We refer to this process with the terms **forward-simulate** or **unroll**.

![closed-loop](../../docs/images/planning/closed-loop.svg)


## What can we use closed-loop evaluation for?
Closed-loop is crucial to asses a model's capabilities before deploying it in the real world. **Ideally, we would test the model on the road in the real world**. However, this is clearly very expensive and scales poorly. Forward-simulation is an attempt to evaluate the system in a setting which is as close as possible to a real road test on the same route.

Differently from open-loop, the model is now in full control of the SDV and predictions are used to compute the future location of the SDV.

## Is closed-loop evaluation enough?
Closed-loop evaluation is an important step forward towards evaluating how our policy would perform on the road. However, it still has some limitations.

The critical one is the **non-reactivity of other traffic participants**. In fact, while the SDV is now controlled by our policy, other actors are still being replayed from the original log. In this setting, a chasing car will not slow down if our policy choses a different speed profile for the SDV, resulting in a rear collision that wouldn't obviously occur in the real world.

For this reason, closed-loop evaluation is only accurate for the first few seconds of forward simulation. This can be mitigated when enough data exists for the task.

### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from prettytable import PrettyTable

from l5kit.configs import load_config_data
from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import EgoDataset
from l5kit.rasterization import build_rasterizer

from l5kit.simulation.dataset import SimulationConfig
from l5kit.simulation.unroll import ClosedLoopSimulator
from l5kit.cle.closed_loop_evaluator import ClosedLoopEvaluator, EvaluationPlan
from l5kit.cle.metrics import (CollisionFrontMetric, CollisionRearMetric, CollisionSideMetric,
                               DisplacementErrorL2Metric, DistanceToRefTrajectoryMetric)
from l5kit.cle.validators import RangeValidator, ValidationCountingAggregator

from l5kit.visualization.visualizer.zarr_utils import simulation_out_to_visualizer_scene
from l5kit.visualization.visualizer.visualizer import visualize
from bokeh.io import output_notebook, show
from l5kit.data import MapAPI

from collections import defaultdict
import os

## Prepare data path and load cfg

By setting the `L5KIT_DATA_FOLDER` variable, we can point the script to the folder where the data lies.

Then, we load our config file with relative paths and other configurations (rasteriser, training params...).

In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = "/mnt/scratch/v_liuhaolan/l5kit_data"
dm = LocalDataManager(None)
# get config
cfg = load_config_data("./goal-config.yaml")
print(cfg)
#cfg = load_config_data("./nmp-config.yaml")

In [ ]:
!echo $PWD

In [ ]:

#model_path = "/mnt/home/v_liuhaolan/haolan/l5kit/lyft_model/planning_model_20201208.pt"
#model_path = "/mnt/home/v_liuhaolan/haolan/l5kit/nmp/planning_goalmap_focal_1_40000.pt"
#model_path = "/mnt/home/v_liuhaolan/haolan/l5kit/nmp/ckpt/planning_1_augmented_fixed.pt"
#model_path = "/mnt/home/v_liuhaolan/haolan/l5kit/nmp/gaussian/balanced/planning_goalmap_4_5000.pt"
#model_path="/mnt/home/v_liuhaolan/haolan/l5kit/nmp/gaussian/new/planning_0_15000_mse_weighted_lr4.pt"

#model_path="/mnt/home/v_liuhaolan/haolan/l5kit/nmp/gaussian/new/planning_2_5000_mse_weighted_lr4.pt" # pretty good!
#planning_0_20000_mse_weighted_lr4.pt" # a model that drives a bit aggressive 
model_path="/mnt/home/v_liuhaolan/haolan/l5kit/nmp/gaussian/new/planning_0_35000_mse_balanced.pt"
#model_path="/mnt/home/v_liuhaolan/haolan/l5kit/nmp/gaussian/new/planning_1_20000_mse_weighted_0.4.pt"
#model_path="/mnt/home/v_liuhaolan/haolan/l5kit/nmp/gaussian/optimized/planning_goalmap_balanced_0_20000.pt"
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



import sys
from torch import nn
sys.path.insert(0, '/mnt/home/v_liuhaolan/haolan/l5kit/nmp')
rasterizer = build_rasterizer(cfg, dm)

from model.goal_map import RasterizedGaussianGoalMapWithUNet
model = RasterizedGaussianGoalMapWithUNet(
        model_arch="resnet50",#cfg["model_params"]["model_architecture"],
        num_input_channels=rasterizer.num_channels(),
        num_targets=3 * cfg["model_params"]["future_num_frames"],  # X, Y, Yaw * number of future states,
        weights_scaling= [1., 1., 1.],
#        criterion=nn.L1Loss(reduction="none")
         criterion=torch.nn.HuberLoss(reduction="none"),
        # num_history = (cfg["model_params"]["history_num_frames"] + 1)*2 ,
        num_mlp_hidden = 64
        )

"""
from model.goal_map import RasterizedGaussianGoalMapWithUNet
model = RasterizedGaussianGoalMapWithUNet(
        model_arch="resnet50",#cfg["model_params"]["model_architecture"],
        num_input_channels=rasterizer.num_channels(),
        num_targets=3 * cfg["model_params"]["future_num_frames"],  # X, Y, Yaw * number of future states,
        weights_scaling= [1., 1., 1.],
#        criterion=nn.L1Loss(reduction="none")
         criterion=torch.nn.HuberLoss(reduction="none"),
        # num_history = (cfg["model_params"]["history_num_frames"] + 1)*2 ,
        num_mlp_hidden = 64
        )




model_path = "/mnt/home/v_liuhaolan/haolan/l5kit/nmp/planning_goalmap_0_100_early_stopping.pt"

from model.goal_map import RasterizedGoalMapAdaptive
model = RasterizedGoalMapAdaptive(
        model_arch="unet",
        num_input_channels=rasterizer.num_channels(),
        num_targets=3 * cfg["model_params"]["future_num_frames"],  # X, Y, Yaw * number of future states,
        weights_scaling= [1., 1., 1.],
#        criterion=nn.L1Loss(reduction="none")
         criterion=torch.nn.HuberLoss(reduction="none"),
        # num_history = (cfg["model_params"]["history_num_frames"] + 1)*2 ,
        num_mlp_hidden = 64
        )

#from l5kit.planning.rasterized.model import RasterizedPlanningModel
from model.models import RasterizedImitationModel
model = RasterizedImitationModel(
        model_arch="resnet50",
        num_input_channels=rasterizer.num_channels(),
        num_targets=3 * cfg["model_params"]["future_num_frames"],  # X, Y, Yaw * number of future states,
        weights_scaling= [1., 1., 1.],
        criterion=nn.L1Loss(reduction="none")
#         criterion=torch.nn.HuberLoss(reduction="none"),
        )





from model.models import RasterizedGoalPlanning
model = RasterizedGoalPlanning(
        model_arch="resnet50",
        num_input_channels=rasterizer.num_channels(),
        num_targets=3 * cfg["model_params"]["future_num_frames"],  # X, Y, Yaw * number of future states,
        weights_scaling= [1., 1., 1.],
#        criterion=nn.L1Loss(reduction="none")
         criterion=torch.nn.HuberLoss(reduction="none"),
        num_history = (cfg["model_params"]["history_num_frames"] + 1)*2 ,
        num_mlp_hidden = 64
)

from model.models import RasterizedTNT
model = RasterizedTNT(
        model_arch="resnet50",
        num_input_channels=rasterizer.num_channels(),
        num_targets=3 * cfg["model_params"]["future_num_frames"],  # X, Y, Yaw * number of future states,
        weights_scaling= [1., 1., 1.],
#        criterion=nn.L1Loss(reduction="none")
         criterion=torch.nn.HuberLoss(reduction="none"),
        num_mlp_hidden = 128
        )

from model.models import RasterizedImitationModel
model = RasterizedImitationModel(
        model_arch="resnet50",
        num_input_channels=rasterizer.num_channels(),
        num_targets=3 * cfg["model_params"]["future_num_frames"],  # X, Y, Yaw * number of future states,
        weights_scaling= [1., 1., 1.],
        criterion=nn.L1Loss(reduction="none")
        )


from model.models import RasterizedTNTWithHistory
model = RasterizedTNTWithHistory(
        model_arch="resnet50",
        num_input_channels=rasterizer.num_channels(),
        num_targets=3 * cfg["model_params"]["future_num_frames"],  # X, Y, Yaw * number of future states,
        weights_scaling= [1., 1., 1.],
#        criterion=nn.L1Loss(reduction="none")
         criterion=torch.nn.HuberLoss(reduction="none"),
        num_mlp_hidden = 128
        )
"""
#model = torch.load(model_path)
model.load_state_dict(torch.load(model_path))


#model = torch.load(model_path)
model = model.to(device)
model = model.eval()
torch.set_grad_enabled(False)

print(model)

## Load the model

In [ ]:
from l5kit.kinematic import AckermanPerturbation
from l5kit.random import GaussianRandomGenerator

from l5kit.geometry import transform_points
from l5kit.visualization import TARGET_POINTS_COLOR, draw_trajectory

mean = np.array([0.0, 0.0, 0.0])  # lateral, longitudinal and angular
std = np.array([0.5, 1.5, np.pi / 6])

#mean = np.array([0.0, 0.0, 1.0])  # lateral, longitudinal and angular
#std = np.array([0.5, 3.5, np.pi / 6])

perturb_prob = cfg["train_data_loader"]["perturb_probability"]


perturbation = AckermanPerturbation(
        random_offset_generator=GaussianRandomGenerator(mean=mean, std=std), perturb_prob=perturb_prob)



# rasterisation and perturbation
rasterizer = build_rasterizer(cfg, dm)

# ===== INIT DATASET

#eval_cfg = cfg["train_data_loader"]
eval_cfg = cfg["val_data_loader"]

rasterizer = build_rasterizer(cfg, dm)
eval_zarr = ChunkedDataset(dm.require(eval_cfg["key"])).open()
eval_dataset = EgoDataset(cfg, eval_zarr, rasterizer)
print(eval_dataset)



# before leaving, ensure perturb_prob is correct
perturbation.perturb_prob = perturb_prob
print(perturb_prob)

In [ ]:
from l5kit.kinematic import AckermanPerturbation
from l5kit.random import GaussianRandomGenerator

from l5kit.geometry import transform_points
from l5kit.visualization import TARGET_POINTS_COLOR, draw_trajectory

mean = np.array([0.0, 0.0, 0.0])  # lateral, longitudinal and angular
std = np.array([1.5, 1.5, np.pi / 6])

#mean = np.array([0.0, 0.0, 1.0])  # lateral, longitudinal and angular
#std = np.array([0.5, 3.5, np.pi / 6])

perturb_prob = 1

perturbation = AckermanPerturbation(
        random_offset_generator=GaussianRandomGenerator(mean=mean, std=std), perturb_prob=perturb_prob)



# rasterisation and perturbation
rasterizer = build_rasterizer(cfg, dm)

# ===== INIT DATASET

#eval_cfg = cfg["train_data_loader"]
train_cfg = cfg["train_data_loader"]

rasterizer = build_rasterizer(cfg, dm)
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
train_dataset = EgoDataset(cfg, train_zarr, rasterizer)#, perturbation=perturbation)
print(train_dataset)



# before leaving, ensure perturb_prob is correct
perturbation.perturb_prob = perturb_prob
print(perturb_prob)

In [ ]:
# Create an input tensor image for your model..
"""
im_ego = rasterizer.to_rgb(data_ego["image"].transpose(1, 2, 0))
print(data_ego["image"].shape)
target_positions = transform_points(data_ego["target_positions"], data_ego["raster_from_agent"])
draw_trajectory(im_ego, target_positions, TARGET_POINTS_COLOR)
plt.imshow(im_ego)
plt.axis('off')
plt.show()
"""

In [ ]:
"""

for j in range(0,20000, 8000):
    # plot same example with and without perturbation
    for perturbation_value in [1, 0]:
        perturbation.perturb_prob = perturbation_value

        data_ego = eval_dataset[j]
        im_ego = rasterizer.to_rgb(data_ego["image"].transpose(1, 2, 0))
        print(data_ego["image"].shape)
        target_positions = transform_points(data_ego["target_positions"], data_ego["raster_from_agent"])
        draw_trajectory(im_ego, target_positions, TARGET_POINTS_COLOR)
        plt.imshow(im_ego)
        plt.axis('off')
        plt.show()

# before leaving, ensure perturb_prob is correct
perturbation.perturb_prob = perturb_prob

"""

In [ ]:


from torch.utils.data import DataLoader

eval_dataloader = DataLoader(eval_dataset, shuffle=True, batch_size=16)
#eval_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)


data_ego = next(iter(eval_dataloader))

#print(data_ego)

print(data_ego["image"][0].shape)

evice = ("cuda:0")
data_ego_cuda = {k: v.to(device) for k, v in data_ego.items()}

model.eval()
traj = model(data_ego_cuda)
#print(traj['positions'])

im_ego = rasterizer.to_rgb((data_ego["image"][0]).numpy().transpose(1, 2, 0))
target_positions = transform_points(data_ego["target_positions"][0].numpy(), data_ego["raster_from_agent"][0].numpy())

# weird hack
im_ego = np.ascontiguousarray(im_ego, dtype=np.uint8)

#draw_trajectory(im_ego, target_positions, TARGET_POINTS_COLOR)
target_positions = transform_points(traj['positions'].cpu().numpy(), data_ego["raster_from_agent"][0].numpy())

#draw_trajectory(im_ego, (target_positions).astype(np.uint8), TARGET_POINTS_COLOR)

#target_positions_pixels = transform_points(data["target_positions"], data["raster_from_agent"])

import cv2
xy_points = data_ego["goal_pixel"][0]
cv2.circle(im_ego, (int(xy_points[0]),int(xy_points[1])), radius=1, color=(255,0,0))

print(xy_points)

plt.imshow(im_ego)
plt.axis('off')
plt.show()


In [ ]:
# grid heatmap
#heatmap = traj["heatmap"][0].view(11,28).cpu().numpy()
heatmap = traj["heatmap"][0].view(112,112).cpu().numpy()


plt.imshow(heatmap, cmap='hot')

plt.show()

heatmap = traj["heatmap"][0][28:84,28:84].view(56,56).cpu().numpy()


plt.imshow(heatmap, cmap='hot')

plt.show()


In [ ]:
# grid heatmap full
heatmap = traj["heatmap"][0].view(112,112).cpu().numpy()

plt.imshow(heatmap, cmap='hot')
pg = traj["predicted_goal"][0]
print(pg)
plt.show()

In [ ]:
#heatmap = data_ego["gt_heatmap_full"][0].view(112,112).cpu().numpy()

import torch
def gaussian_tensor(xL, yL, H, W, device, sigma=5):

    #channel = torch.zeros((H*W), dtype=torch.float)#.to("cuda:0")
    x = torch.arange(H).view(H,1).repeat(1,W).to(device)
    y = torch.arange(W).repeat(H,1).to(device)
    channel = torch.exp((-((x - xL) ** 2 + (y - yL) ** 2) / (2 * sigma ** 2)))

    channel = channel.reshape(H, W)

    return channel

def batch_gaussian_tensor(xL, yL, H, W, device, sigma=5 ):

 #channel = torch.zeros((H*W), dtype=torch.float)#.to("cuda:0")
    batch_size = xL.shape[0]
    channel = torch.zeros(batch_size, H, W).to(device)

    x = torch.arange(H).view(H,1).repeat(1,W).to(device)
    y = torch.arange(W).repeat(H,1).to(device)
    
    for i in range(batch_size):
        channel[i] = torch.exp((-((x - xL[i]) ** 2 + (y - yL[i]) ** 2) / (2 * sigma ** 2)))

    channel = channel.reshape(batch_size, H, W)

    return channel


gt_goal_positions_pixels = data_ego["goal_pixel"]

#print(gt_goal_positions_pixels)
gt_goal_positions_pixels = gt_goal_positions_pixels.to("cuda:0")

from time import time
start = time()
heatmap = batch_gaussian_tensor(gt_goal_positions_pixels[:,1], gt_goal_positions_pixels[:,0], 112, 112,sigma=2,device="cuda:0")
print(time()-start)

heatmap = heatmap[0].cpu().numpy().squeeze()
plt.imshow(heatmap, cmap='hot')

plt.show()
pg = traj["predicted_goal"][0]
print(pg)

In [ ]:
heatmap = traj["heatmap"][0].view(112,112).cpu().numpy()

plt.imshow(heatmap, cmap='hot')

a = np.argmax(heatmap)
print([a//112, a%112])

plt.show()

## Load the evaluation data
Differently from training and open loop evaluation, this setting is intrinsically sequential. As such, we won't be using any of PyTorch's parallelisation functionalities.

In [ ]:
#device = 'cpu'

## Define some simulation properties
We define here some common simulation properties such as the length of the simulation and how many scene to simulate.

**NOTE: these properties have a significant impact on the execution time. We suggest you to increase them only if your setup includes a GPU**

In [ ]:
num_scenes_to_unroll = 30
num_simulation_steps = 130

# Closed-loop simulation

We define a closed-loop simulation that drives the SDV for `num_simulation_steps` steps while using the log-replay agents.

Then, we unroll the selected scenes.
The simulation output contains all the information related to the scene, including the annotated and simulated positions, states, and trajectories of the SDV and the agents.  
If you want to know more about what the simulation output contains, please refer to the source code of the class `SimulationOutput`.

In [ ]:
# ==== DEFINE CLOSED-LOOP SIMULATION
sim_cfg = SimulationConfig(use_ego_gt=False, use_agents_gt=True, disable_new_agents=True,
                           distance_th_far=500, distance_th_close=50, num_simulation_steps=num_simulation_steps,
                           start_frame_index=0, show_info=True)

sim_loop = ClosedLoopSimulator(sim_cfg, train_dataset, device, model_ego=model, model_agents=None, keys_to_exclude={"image","loss","loss1","loss2","goal_score","regression","heatmap"})

#sim_loop = ClosedLoopSimulator(sim_cfg, eval_dataset, device, model_ego=model, model_agents=None, keys_to_exclude={"image","loss","loss1","loss2","goal_score","regression","heatmap"})
#sim_loop = ClosedLoopSimulator(sim_cfg, eval_dataset, device, model_ego=None, model_agents=None, keys_to_exclude={"image","loss","motion_loss","target_loss"})

In [ ]:
# ==== UNROLL
import time
start = time.time()

#scenes_to_unroll = list(range(0, len(eval_zarr.scenes), len(eval_zarr.scenes)//num_scenes_to_unroll))
scenes_to_unroll = list(range(0, len(train_zarr.scenes), len(train_zarr.scenes)//num_scenes_to_unroll))


sim_outs = sim_loop.unroll(scenes_to_unroll)

print(time.time()-start)

In [ ]:
print(train_dataset)

# Closed-loop metrics
In this setting **metrics are particularly challenging**.
In fact, we would like to penalise some of the simulation drift (e.g. going off road or in the opposite lane) while at the same time allow others (e.g. different speed profiles).


### Collisions
Our SDV should avoid collisions with other agents. In this example, we won't distinguish between collisions caused by non-reactivity of other agents and actual collisions, and we will simply report them all categorised by where they occurred (front, rear and side with respect to the AV).

However, if we only considered collisions, our SDV could pass all tests by simply driving off-road or in a different lane.


### Distance from reference trajectory
To address the issue presented above, we require our SDV to loosely stick to the original trajectory in the data. By setting the right threshold on the distance we can allow for different speed profiles and small steerings, while pensalising large deviations like driving off-road.

We can do so by computing the distance between the simulated trajectory and the annotated one in world coordinates.


### Displacement error (L2)
In addition, we can measure the displacement between each point of the simulated trajectory and the corresponding annotated one in world coordinates.

With this metric, we can spot large deviations between the speed of the simulated policy and the annotated one.

In [ ]:
metrics = [DisplacementErrorL2Metric(),
           DistanceToRefTrajectoryMetric(),
           CollisionFrontMetric(),
           CollisionRearMetric(),
           CollisionSideMetric()]

validators = [RangeValidator("displacement_error_l2", DisplacementErrorL2Metric, max_value=30),
              RangeValidator("distance_ref_trajectory", DistanceToRefTrajectoryMetric, max_value=4),
              RangeValidator("collision_front", CollisionFrontMetric, max_value=0),
              RangeValidator("collision_rear", CollisionRearMetric, max_value=0),
              RangeValidator("collision_side", CollisionSideMetric, max_value=0)]

intervention_validators = ["displacement_error_l2",
                           "distance_ref_trajectory",
                           "collision_front",
                           "collision_rear",
                           "collision_side"]

cle_evaluator = ClosedLoopEvaluator(EvaluationPlan(metrics=metrics,
                                                   validators=validators,
                                                   composite_metrics=[],
                                                   intervention_validators=intervention_validators))

# Quantitative evaluation

We can now compute the metric evaluation, collect the results and aggregate them.

In [ ]:
cle_evaluator.evaluate(sim_outs)
validation_results = cle_evaluator.validation_results()
agg = ValidationCountingAggregator().aggregate(validation_results)
cle_evaluator.reset()

## Reporting errors from the closed-loop

We can now report the metrics and plot them.

In [ ]:
fields = ["metric", "value"]
table = PrettyTable(field_names=fields)

values = []
names = []

for metric_name in agg:
    table.add_row([metric_name, agg[metric_name].item()])
    values.append(agg[metric_name].item())
    names.append(metric_name)

print(table)

plt.bar(np.arange(len(names)), values)
plt.xticks(np.arange(len(names)), names, rotation=60, ha='right')
plt.show()

# Qualitative evaluation

## Visualise the closed-loop

We can visualise the scenes we have obtained previously. 

**The policy is now in full control of the SDV as this moves through the annotated scene.**

In [ ]:
output_notebook()
mapAPI = MapAPI.from_cfg(dm, cfg)
for sim_out in sim_outs[-10:]: # for each scene
    vis_in = simulation_out_to_visualizer_scene(sim_out, mapAPI)
    show(visualize(sim_out.scene_id, vis_in))

In [ ]:
output_notebook()
mapAPI = MapAPI.from_cfg(dm, cfg)
for sim_out in sim_outs[-5:]: # for each scene
    vis_in = simulation_out_to_visualizer_scene(sim_out, mapAPI)
    show(visualize(sim_out.scene_id, vis_in))

In [ ]:
from bokeh.io import output_file, show

for sim_out in sim_outs[-5:]: # for each scene
    vis_in = simulation_out_to_visualizer_scene(sim_out, mapAPI)
    q = (visualize(sim_out.scene_id, vis_in))
    output_file("./bokeh.png")
    show(q)
    break

# Pre-trained model results

We include here the unroll of one scene using one of our pre-trained model. The controlled SDV can stick to the correct lane and stops successfully for a red light. 

Comparing this result with the one from the [open-loop notebook](./open_loop_test.ipynb) we can notice some differences in the speed profile chosen by the model.


![SegmentLocal](../../docs/images/planning/out_9_closed.gif "segment")
